In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
test=pd.read_csv('test.csv')

In [5]:
test.shape

(153164, 2)

In [6]:
test.columns

Index(['id', 'comment_text'], dtype='object')

In [7]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [8]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [9]:
xtrain=train['comment_text'].values
ytrain=train[['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']].values
xtest=test['comment_text'].values

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [13]:
type(xtrain)

numpy.ndarray

In [17]:
xtrain[1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [21]:
tokenizer=Tokenizer(num_words=5000,oov_token='<oov>')
tokenizer.fit_on_texts(list(xtrain))
xtrainseq=tokenizer.texts_to_sequences(xtrain)
xtestseq=tokenizer.texts_to_sequences(xtest)
xtrainpadded=pad_sequences(xtrainseq,maxlen=60,padding='post',truncating='post')
xtestpadded=pad_sequences(xtestseq,maxlen=60,padding='post',truncating='post')

In [22]:
print(xtrain)

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead o

In [23]:
print(xtrainpadded)

[[ 689   76    2 ...    0    0    0]
 [   1   53 2636 ...    0    0    0]
 [ 413  438   74 ...    0    0    0]
 ...
 [   1    1    1 ...    0    0    0]
 [   5   12  575 ...    0    0    0]
 [   5    8  135 ...    0    0    0]]


In [28]:
xtrain.shape

(159571,)

In [24]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(5000,15,input_length=60),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(60,activation='relu'),
    tf.keras.layers.Dense(6,activation='sigmoid')
])

In [25]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [26]:
model.fit(xtrainpadded,ytrain,batch_size=32,epochs=1,validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
143613/143613 [==============================] - 195s 1ms/sample - loss: 0.0691 - accuracy: 0.9781 - val_loss: 0.0555 - val_accuracy: 0.9808
